<a href="https://colab.research.google.com/github/rkwi/fenics-notebooks/blob/master/Notebooks/stokes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This program solves the steady incompressible stokes equations in a 2D lid driven cavity.

# Setup FEniCS

In [0]:
from google.colab import files

import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    from dolfin import *; from mshr import *
    
import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot 
import time

import os, sys, shutil

dolfin_version = dolfin.__version__
print ('dolfin version:', dolfin_version)

!rm -rf * # clean up all files
# Useful commands
# Remove an empty folder      : os.rmdir("my_results")
# Remove a folder with files  : shutil.rmtree("results")
# Make a folder               : os.mkdir("my_results")
# Runtime/Change_runtime_type/Python3

from fenics import *

# Create mesh and define function spaces

We create a triangle mesh on a unit square and use the Taylor Hood element pair as our function spaces. 

In [0]:
mesh = UnitSquareMesh(10, 10)
P2 = VectorElement('CG', triangle, 2, dim=2)
P1 = FiniteElement('CG', triangle, 1)
TH = P2 * P1
W = FunctionSpace(mesh, TH)

In older versions, the Taylor Hood element pair can be defined using the following way. Note that this will not work in earlier versions because we can no longer take the product of two function spaces using the product operator.

In [0]:
V = VectorFunctionSpace(mesh,'CG',2, dim=2)
Q = FunctionSpace(mesh, 'CG', 1)
W = V*Q

# Define boundaries

Because of rounding errors, we need to use tolerances to compare real numbers. Tolerances as low as $3 \times 10^{-16}$ can be used, and we can easily use the near function to implement this.

In [0]:
lid = 'near(x[1],1)'
walls_and_base = 'near(x[0],0) || near(x[0],1) || near(x[1],0)'
origin = 'near(x[0],0) && near(x[1],0)'

In older versions, the near function may not be available. In that case, we use the constant DOLFIN_EPS.

In [0]:
lid = 'x[1] > 1.0 - DOLFIN_EPS'
walls_and_base = 'x[0] < DOLFIN_EPS || x[0] > 1.0 - DOLFIN_EPS || x[1] < DOLFIN_EPS'
origin = 'x[0] < DOLFIN_EPS && x[1] < DOLFIN_EPS'

# Define boundary conditions

In [0]:
bc_lid = DirichletBC(W.sub(0), (1,0), lid)
bc_noslip = DirichletBC(W.sub(0), (0,0), walls_and_base)
bc_p = DirichletBC(W.sub(1), 0, origin, 'pointwise')
bc = [bc_lid, bc_noslip, bc_p]

# Define variational problem

Note that we use TrialFunctions and TestFunctions instead of TrialFunction and TestFunction. 

In [0]:
(u,p) = TrialFunctions(W)
(v,q) = TestFunctions(W)
f = Constant((0,0))
a = inner(grad(u), grad(v))*dx - p*div(v)*dx + div(u)*q*dx
L = dot(f, v)*dx

# Compute solution

In [0]:
w = Function(W)
solve(a == L, w, bc)
(u,p) = w.split()

# Plot and save solution

In [0]:
plot(u)
vtkfile = File('solution_stokes.pvd')
vtkfile << u

# Get and view current directory

In [0]:
import os
print(os.getcwd())
print(os.listdir())

# Download solution files

In [0]:
from google.colab import files
files.download('solution_stokes.pvd')
files.download('solution_stokes000000.vtu')